In [23]:
import os
import pandas as pd
import io

import google.auth
import random
import math
from Google import Create_Service
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

In [24]:

rekeningDf = pd.read_csv('rekening_petugas_clean_chory.csv', sep=';')

In [25]:
splittedDf = rekeningDf.head()
splittedDf.iloc[0:5]

,No,Timestamp,KECAMATAN,DESAKELURAHAN DI KECAMATAN AMBARAWA,NAMA DUSUN/ DUKUH/LINGKUNGAN,NAMA RT 000 RW 00,NAMA PENUNJUK JALAN,FOTO KTP PENUNJUK JALAN,FOTO NPWP PENUNJUK JALAN JIKA MEMILIKI,FOTO BUKU TABUNGAN\nHALAMAN NAMA & NOMOR REKENING ATAS NAMA SENDIRI PENUNJUK JALAN,NAMA REKENING SESUAI BUTAB,NOREK,NAMA BANK,KET BUTAB,SLS dinyatakan selesai (Mb Mery),Tanggal pengajuan ke Dwi (STIS),Tanggal pembayaran (Dwi)
0,1,10/15/2022 10:58:17,AMBARAWA,BEJALEN,Bejalen,RT 010 RW 04,ISTIYARNO,https://drive.google.com/open?id=1zIYsbbiQkXOu...,NaN,https://drive.google.com/open?id=1VDCUNE8CjIgb...,ISTIYARNO,608701016774532,BRI,SESUAI,NaN,16-Nov-22,NaN
1,2,10/28/2022 18:45:59,AMBARAWA,KRANGGAN,JAGALAN,RT 002 RW 07,PRIHANTINI,https://drive.google.com/open?id=1d-OI04soKrB3...,NaN,https://drive.google.com/open?id=1LKCVHTfbMr-v...,PRIHANTINI,608601008379530,BRI,SESUAI,NaN,16-Nov-22,NaN
2,3,10/21/2022 10:37:11,AMBARAWA,KRANGGAN,KEPATIHAN,RT 002 RW 01,HERI NURUL SUSANTO,https://drive.google.com/open?id=1jeYck0NaGCCz...,https://drive.google.com/open?id=1NVNV6-GuZTXj...,https://drive.google.com/open?id=1OBZcF9KJbRE7...,HERI NURUL SUSANTO,51301027074502,BRI BRITAMA,SESUAI,NaN,16-Nov-22,NaN
3,4,10/21/2022 10:51:37,AMBARAWA,KRANGGAN,LEGOKSARI,RT 001 RW 03,RAHMAN ALI SASMITO,https://drive.google.com/open?id=1l2McnAPXqCGf...,https://drive.google.com/open?id=1czmZaShMUSNz...,https://drive.google.com/open?id=1W6J25ptQYvv2...,RAHMAN ALI SASMITO,608701020201539,BRI SIMPEDES,SESUAI,NaN,16-Nov-22,NaN
4,5,10/21/2022 10:54:24,AMBARAWA,KRANGGAN,LEGOKSARI,RT 002 RW 03,MARTINUS HARI KURNIAWAN,https://drive.google.com/open?id=1FD6Qwhp0qnNu...,NaN,https://drive.google.com/open?id=1LA_6aSgKgKDU...,MARTINUS HARI KURNIAWAN,608601010635532,BRI SIMPEDES,SESUAI,NaN,16-Nov-22,NaN


In [26]:
CLIENT_SECRET_FILE = 'credentials.json'
API_NAME='drive'
API_VERSION='v3'
SCOPES=['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)



credentials.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully


In [27]:
columnToDownload = ['FOTO KTP PENUNJUK JALAN', 'FOTO NPWP PENUNJUK JALAN JIKA MEMILIKI', 'FOTO BUKU TABUNGAN\nHALAMAN NAMA & NOMOR REKENING ATAS NAMA SENDIRI PENUNJUK JALAN']
rootDir = "rekening-chory"


# create foto ktp penunjuk jalan
for col in columnToDownload :
    count=1
    total = len(rekeningDf.index)
    for index, row in rekeningDf.iterrows():
    
        url = row[col]

        if not pd.isna(url):
            id = url.split('=')[1]
            # create the output directory, the downloaded file should be put into individual folder
            outputDir = row['NAMA REKENING SESUAI BUTAB']+'-'+str(row['KECAMATAN'])+'-'+str(row['NOREK'])
            path=os.path.join(rootDir, outputDir)

            if(not os.path.exists(path)):
                os.makedirs(path)
            match col:
                case 'FOTO KTP PENUNJUK JALAN':
                    print('ktp')
                    tempFilename = 'ktp.jpg'
                case 'FOTO NPWP PENUNJUK JALAN JIKA MEMILIKI':
                    print('npwp')
                    tempFilename = 'npwp.jpg' 
                case 'FOTO BUKU TABUNGAN\nHALAMAN NAMA & NOMOR REKENING ATAS NAMA SENDIRI PENUNJUK JALAN':
                    print('butab')
                    tempFilename = 'butab.jpg'
                case _:
                    print('unhandled')

            filename=os.path.join(path, tempFilename)

            # get file using api created before
            request = service.files().get_media(fileId=id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fd=fh, request=request)

            done = False

            while not done:
                status, done = downloader.next_chunk()
                # print('Download progress {0}'.format(status.progress()*100))

            fh.seek(0)

            # save downloaded file to the directory previously defined
            with open(filename, 'wb') as f:
                f.write(fh.read())
                f.close()
            progress=(count/total)*100
            progressPercent = math.ceil(progress*100)/100
            print('Overall progress {0}% ({1}/{2})'.format(progressPercent, count, total))
            count+=1
    
  

ktp
Overall progress 1.57% (1/64)
ktp
Overall progress 3.13% (2/64)
ktp
Overall progress 4.69% (3/64)
ktp
Overall progress 6.25% (4/64)
ktp
Overall progress 7.82% (5/64)
ktp
Overall progress 9.38% (6/64)
ktp
Overall progress 10.94% (7/64)
ktp
Overall progress 12.5% (8/64)
ktp
Overall progress 14.07% (9/64)
ktp
Overall progress 15.63% (10/64)
ktp
Overall progress 17.19% (11/64)
ktp
Overall progress 18.75% (12/64)
ktp
Overall progress 20.32% (13/64)
ktp
Overall progress 21.88% (14/64)
ktp
Overall progress 23.44% (15/64)
ktp
Overall progress 25.0% (16/64)
ktp
Overall progress 26.57% (17/64)
ktp
Overall progress 28.13% (18/64)
ktp
Overall progress 29.69% (19/64)
ktp
Overall progress 31.25% (20/64)
ktp
Overall progress 32.82% (21/64)
ktp
Overall progress 34.38% (22/64)
ktp
Overall progress 35.94% (23/64)
ktp
Overall progress 37.5% (24/64)
ktp
Overall progress 39.07% (25/64)
ktp
Overall progress 40.63% (26/64)
ktp
Overall progress 42.19% (27/64)
ktp
Overall progress 43.75% (28/64)
ktp
Overal

In [31]:
rekeningDf.iloc[330]

Timestamp                                                                                            17/11/2022 07:30
Wilayah Kerja (Kecamatan)                                                                                     Geragai
Apakah anda memiliki rekening yang digunakan untuk kegiatan BPS?                                             Memiliki
Nama Petugas (sesuai KTP)                                                                                         NaN
Nama ibu kandung (sesuai KTP)                                                                                     NaN
Foto KTP                                                                                                          NaN
Nomor Telepon                                                                                                     NaN
Nomor Rekening Aktif                                                                                 ?710401015097538
Nama Petugas (Sesuai Buku Rekening)                     

In [39]:
len(os.listdir('rekening'))

353